# RIS Phase Hopping for Ultra-Reliable Communications

_Autor:_ Karl-Ludwig Besser, Technische Universität Braunschweig

Este cuaderno forma parte de los artículos "Reconfigurable Intelligent Surface Phase Hopping for Ultra-Reliable Communications" (K.-L. Besser y E. Jorswieck, IEEE Transactions on Wireless Communications, vol. 21, no. 11, pp. 9082–9095, noviembre de 2022. [doi:10.1109/TWC.2022.3172760](https://doi.org/10.1109/TWC.2022.3172760), [arXiv:2107.11852](https://arxiv.org/abs/2107.11852)) y "Artificial Fast Fading from Reconfigurable Surfaces Enables Ultra-Reliable Communications" (SPAWC 2021 [doi:10.1109/SPAWC51858.2021.9593269](https://doi.org/10.1109/SPAWC51858.2021.9593269)).  
Si utilizas algún trabajo de este cuaderno, por favor, cita el artículo mencionado anteriormente.


> If you are not familiar with Jupyter notebooks: The easiest way to use this notebook interactively, is to hit `Kernel --> Restart & Run All` in the menu. This will execute all cells and enable the interactive elements of the plots.  
> Alternatively, you can execute the cells one by one using Shift+Return

In [ ]:
# Ejecutar celda en Google Colab para instalar las dependencias
!pip install -r requirements.txt

In [ ]:
import numpy as np
from scipy import stats
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual

In [ ]:
from phases import gains_constant_phase, rvs_channel_phases, rvs_ris_phases, rvs_ris_phases_quant

In [ ]:
# Soporte para widgets en google colab
from google.colab import output
output.enable_custom_widget_manager()

## Fases Constantes de RIS

Primero, consideramos fases constantes de RIS _sin_ saltos de fase.

Se observa que la capacidad de $\varepsilon$-interrupción es cercana a cero para valores de $\varepsilon$ cercanos a cero (para todos los valores de $N$).

In [ ]:
def constant_ris_phases(num_samples=50000, num_elements=[5, 10, 20, 50]):
    fig, axs = plt.subplots()
    def update(show_approx=True):
        axs.clear()
        axs.set_xlabel("Tasa $R$")
        axs.set_ylabel("Probabilidad de Interrupción $\\varepsilon$")
        for _num_elements in num_elements:
            channel_realizations = rvs_channel_phases(_num_elements, num_samples)
            const_phase = gains_constant_phase(channel_realizations)
            capac_const_phase = np.log2(1 + const_phase)
            _hist = np.histogram(capac_const_phase, bins=100)
            _r_ax = np.linspace(min(capac_const_phase)*.9, max(capac_const_phase)*1.1, 1000)
            cdf_hist = stats.rv_histogram(_hist).cdf(_r_ax)
            axs.plot(_r_ax, cdf_hist, label="CDF Empírica N={:d}".format(_num_elements))
            if show_approx:
                cdf_appr = 1. - np.exp(-(2**_r_ax-1)/_num_elements)  # N --> oo, para la suma
                axs.plot(_r_ax, cdf_appr, '--', label="Aproximado N={:d}".format(_num_elements))
        axs.legend()
    interact(update, show_approx=True)


In [ ]:
constant_ris_phases()

## Saltos de Fase en RIS

A continuación, consideramos saltos de fase en RIS con fases $\theta_i$ variadas aleatoriamente, $i=1, \dots{}, N$.
Todas las $\theta_i$ son variables aleatorias independientes e idénticamente distribuidas con una distribución uniforme en $[0, 2\pi]$.

Se observa que la probabilidad de interrupción es una función escalonada cuando se emplean saltos de fase.

In [ ]:
def n_element_fast_fading(num_samples_slow=1000, num_samples_fast=5000):
    fig, axs = plt.subplots()
    def update(num_elements=3):
        axs.clear()
        channel_realizations = rvs_channel_phases(num_elements, num_samples_slow)
        channel_realizations = np.tile(channel_realizations, (num_samples_fast, 1, 1))
        ris_phases = rvs_ris_phases(num_elements, num_samples_slow, num_samples_fast, copula="indep")
        channel_realizations = channel_realizations + ris_phases
        const_phase = gains_constant_phase(channel_realizations)
        capac_const_phase = np.log2(1 + const_phase)
        expect_capac = np.mean(capac_const_phase, axis=0)
        _hist = np.histogram(expect_capac, bins=100)
        _r_ax = np.linspace(0, 3, 1000)
        cdf_hist = stats.rv_histogram(_hist).cdf(_r_ax)
        axs.plot(_r_ax, cdf_hist)
        axs.set_title("Saltos de Fase en RIS con N={:d} Elementos RIS".format(num_elements))
        axs.set_xlabel("Tasa $R$")
        axs.set_ylabel("Probabilidad de Interrupción $\\varepsilon$")
    interact_manual(update, num_elements=(2, 10, 1))


In [ ]:
n_element_fast_fading()

### Fases Cuantizadas

Anteriormente, se han considerado fases continuas de RIS en el conjunto $[0, 2\pi]$.
En lo que sigue, asumimos que solo está disponible un conjunto discreto de posibles valores de fase, es decir,
$$\theta_i\in\mathcal{Q}=\left\{k\frac{2\pi}{K}\;\middle|\; k=0,\dots{},K-1\right\}.$$

El siguiente gráfico muestra un ejemplo con $N=2$ y $K=2$.

In [ ]:
def quantized_two_phases_two_elements(num_samples_slow=1000, num_samples_fast=5000):
    fig, axs = plt.subplots()
    num_elements = 2
    dependency = "indep"
    channel_realizations = rvs_channel_phases(num_elements, num_samples_slow)
    channel_realizations = np.tile(channel_realizations, (num_samples_fast, 1, 1))
    ris_phases = rvs_ris_phases_quant(num_elements, num_samples_slow, num_samples_fast,
                                      copula=dependency, K=2)
    total_phases = channel_realizations + ris_phases
    const_phase = gains_constant_phase(total_phases)
    capac_const_phase = np.log2(1 + const_phase)
    expect_capac = np.mean(capac_const_phase, axis=0)
    _hist = np.histogram(expect_capac, bins=100)
    _r_ax = np.linspace(0, 3, 1000)
    cdf_hist = stats.rv_histogram(_hist).cdf(_r_ax)
    zoc = 0.5*np.log2(5)
    axs.plot(_r_ax, cdf_hist, label="ECDF")
    axs.vlines(zoc, 0, 1, 'r', label="ZOC -- Exacto")
    axs.legend()
    axs.set_title("Artificial Fast Fading con N={:d} Elementos RIS\nFases Cuantizadas con 2 Pasos de Cuantización".format(num_elements))
    axs.set_xlabel("Tasa $R$")
    axs.set_ylabel("Probabilidad de Interrupción $\\varepsilon$")


In [ ]:
quantized_two_phases_two_elements(num_samples_fast=100000)

A continuación, visualizamos un esquema que permite obtener una ZOC positiva utilizando un RIS con $N=2$ elementos y $K=2$ posibles valores de fase. Cada símbolo se repite $L=2$ veces. La capacidad total se expresa como:

\begin{equation}
\frac{1}{L} \log_2\left(1 + \sum_{l=1}^{L}\left|{\sum_{i=1}^{N}\exp\left(\mathrm{j} \left(\theta_{i,l} + \varphi_i\right)\right)}\right|^2\right)\,.
\end{equation}

Las fases de los elementos RIS se establecen en $\theta_{1,1}=\theta_{1,2}=0$ y $\theta_{2,1}=0$ y $\theta_{2,2}=\pi$. Con esto, podemos lograr un valor constante de la relación señal-ruido resultante:

$$\sum_{l=1}^{2}\left|{\sum_{i=1}^{2}\exp\left(\mathrm{j} \left(\theta_{i,l} + \varphi_i\right)\right)}\right|^2 = 4,$$

independientemente de las realizaciones del canal $\varphi_i$.

In [ ]:
def positive_zoc_repetition():
    fig, (axs, axs2) = plt.subplots(1, 2)
    axs.set_xlim([-2, 2])
    axs.set_ylim([-2, 2])
    axs2.set_xlim([0, 1])
    axs2.set_ylim([0, 5])
    axs2.set_ylabel("Valor Absoluto al Cuadrado")
    plot1 = axs.plot([0, 1], [0, 1], '--')[0]
    plot21 = axs.plot([0, 1], [0, 1], '--')[0]
    plot22 = axs.plot([0, 1], [0, 1], '--')[0]
    plot_abs1 = axs.plot([0, 1], [0, 1], 'o-')[0]
    plot_abs2 = axs.plot([0, 1], [0, 1], 'o-')[0]
    plot2_abs1 = axs2.plot([0, 1], [0, 1], '-', color=plot_abs1.get_color(), lw=4)[0]
    plot2_abs2 = axs2.plot([0, 1], [0, 1], '-', color=plot_abs2.get_color(), lw=4)[0]
    
    def update(phi1=np.pi/4, phi2=0):
        z1 = np.exp(1j*phi1)
        z21 = z1 + np.exp(1j*phi2)
        z22 = z1 + np.exp(1j*(phi2+np.pi))
        plot1.set_data([0, np.real(z1)], [0, np.imag(z1)])
        plot21.set_data([np.real(z1), np.real(z21)], [np.imag(z1), np.imag(z21)])
        plot22.set_data([np.real(z1), np.real(z22)], [np.imag(z1), np.imag(z22)])
        
        plot_abs1.set_data([0, np.real(z21)], [0, np.imag(z21)])
        plot_abs2.set_data([0, np.real(z22)], [0, np.imag(z22)])
        
        plot2_abs1.set_data([0.5, 0.5], [0, np.abs(z21)**2])
        plot2_abs2.set_data([0.5, 0.5], [np.abs(z21)**2, np.abs(z21)**2 + np.abs(z22)**2])
    
    interact(update, phi1=(0, 2*np.pi, .01), phi2=(0, 2*np.pi, .01))


In [ ]:
positive_zoc_repetition()